In [1]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [2]:
symbtr_cols = ['Sira', 'Kod', 'Nota53', 'NotaAE', 'Koma53', 'KomaAE', 
               'Pay', 'Payda', 'Ms', 'LNS', 'Bas', 'Soz1', 'Offset']
savefolder = os.path.join('..', '..', 'symbtr_corrections', 'usul')
symbtrs = [os.path.splitext(s)[0] for s in getFileNamesInDir('../txt', keyword='*.txt')[2]]

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

In [3]:
for ii, symbTr in enumerate(symbtrs):
    print str(ii) + ': ' + symbTr
    
    symbtrtxtfile = os.path.join('../txt', symbTr + '.txt')
    symbtrmu2file = os.path.join('../mu2', symbTr + '.mu2')

    savefile = os.path.join(savefolder, symbTr + '.txt')

    # extract symbtr data
    txtdata = extractor.extract(symbtrtxtfile)[0]
    mu2header = symbtrreader.readMu2Header(symbtrmu2file)[0]

    # merge
    data = extractor.merge(txtdata, mu2header)

    # get the usul variant
    variant = {}
    for vrt in usuldict[data['usul']['symbtr_slug']]['variants']:
        if vrt['mu2_name'] == data['usul']['mu2_name']:
            variant = vrt
            break

    df = pd.read_csv(symbtrtxtfile, sep='\t')
    if not df.iloc[0]['Kod'] == 51:
        # create the usul row
        # 1	51			0	0	zaman	mertebe	0	usul_symbtr_internal_id	0	usul_mu2_name	0
        # 1	51			0	0	6	4	0	90	0	Yürüksemâî (6/4)	0
        usulrow = pd.DataFrame({'Sira':1, 'Kod':51, 'Nota53':'', 'NotaAE':'', 'Koma53':0, 'KomaAE':0,
                                'Pay':int(variant['num_pulses']),'Payda':int(variant['mertebe']),'Ms':0,'Offset':0,
                                'LNS':variant['symbtr_internal_id'],'Bas':0,'Soz1':variant['mu2_name']}, index=[0])
        df2 = pd.DataFrame(df,index=range(1, df.size+1))

        for index, row in df.iterrows():       
            # change null to empty string
            for key, val in row.iteritems():
                if pd.isnull(val):
                    row[key] = ''

            # increase Sira by one so the row indices are consecutive
            row['Sira'] = row['Sira']+1

            # reassign
            df.iloc[index] = row

        df_usul = pd.concat([usulrow, df], ignore_index=True)[symbtr_cols]
        df_usul.to_csv(savefile, sep='\t', index=False, encoding='utf-8')
    else:
        print symbTr + " starts with the usul row"

0: segah--pesrev--devrikebir----neyzen_yusuf_pasa
   Unknown title target.
segah--pesrev--devrikebir----neyzen_yusuf_pasa starts with the usul row
1: sedaraban--pesrev--fahte----tanburi_cemil_bey
   Unknown title target.
sedaraban--pesrev--fahte----tanburi_cemil_bey starts with the usul row
2: ussak--sarki--nimsofyan--yillar_sonra--cahit_deniz
   Unknown title target.
ussak--sarki--nimsofyan--yillar_sonra--cahit_deniz starts with the usul row
3: huseyni--turku--curcuna--coban_kizi--
    48.0: dan   and Le  are in the same measure!
    43.0: dan   and Le  are in the same measure!
    37.0: rür   and Öl are in the same measure!
    32.0: rür   and Öl are in the same measure!
    28.0: rur   and Le  are in the same measure!
    24.0: rur   and Le  are in the same measure!
    19.0: rur   and Dol are in the same measure!
    13.0: rur   and Dol are in the same measure!
huseyni--turku--curcuna--coban_kizi--, 130, VOCAL_SECTION does not start on a measure: 13.5
huseyni--turku--curcuna--coban